# Fase 1 · Preparación inicial del dataset de exoplanetas (KOI)

En esta fase preparamos el conjunto de datos base a partir del catálogo original del NASA Exoplanet Archive (KOI).  
El objetivo es obtener un archivo limpio y coherente que se reutilizará en las siguientes fases (EDA, ingeniería de características, modelado y evaluación).

En concreto, aquí vamos a:
- Cargar el archivo original de exoplanetas.
- Inspeccionar las columnas más relevantes, especialmente `koi_disposition`.
- Filtrar solo los objetos con disposición `CONFIRMED` y `FALSE_POSITIVE`.
- Crear la variable objetivo binaria `is_confirmed`.
- Eliminar columnas administrativas o poco informativas.
- Guardar el resultado como `exoplanet_koi_binary_clean.csv` para usarlo en las fases posteriores.


## 1. Importar librerías básicas

En esta sección se importan las librerías fundamentales para el análisis:

- `pandas` y `numpy` para la manipulación de datos.
- Otras librerías auxiliares que se usarán a lo largo de todas las fases del proyecto.

Estas importaciones son estándar y se reutilizarán en los demás cuadernos.


In [ ]:
import pandas as pd
import numpy as np

print("Versión de pandas:", pd.__version__)

Versión de pandas: 2.2.2


## 2. Definir la ruta del archivo original

Aquí se define la ruta del archivo CSV original descargado del NASA Exoplanet Archive.
El link para descargarlo es el siguiente: "https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=cumulative"
(Descargarlo con todas las filas y todas las columnas, y en formato CSV).  
En este punto el usuario puede:

- Ajustar la ruta según si está trabajando en Google Colab, Jupyter local, etc.
- Verificar el nombre del archivo original para evitar errores de carga.

Esta celda no transforma datos, solo prepara la referencia al archivo fuente.


In [ ]:
file_path = "/content/cumulative_2025.12.01_16.06.39.csv"

# El archivo de la NASA tiene muchas líneas de encabezado que empiezan con '#'
# Por eso usamos comment='#' para que pandas las ignore y lea directamente la tabla.

df_raw = pd.read_csv(file_path, comment='#')

print("Dimensiones del dataset ORIGINAL (df_raw):", df_raw.shape)
df_raw.head()

Dimensiones del dataset ORIGINAL (df_raw): (9564, 83)


,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_vet_stat,koi_vet_date,koi_pdisposition,koi_score,koi_fpflag_nt,...,koi_fwm_srao,koi_fwm_sdeco,koi_fwm_prao,koi_fwm_pdeco,koi_dicco_mra,koi_dicco_mdec,koi_dicco_msky,koi_dikco_mra,koi_dikco_mdec,koi_dikco_msky
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,Done,2018-08-16,CANDIDATE,1.000,0,...,0.430,0.940,-0.00020,-0.00055,-0.010,0.200,0.200,0.080,0.310,0.320
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,Done,2018-08-16,CANDIDATE,0.969,0,...,-0.630,1.230,0.00066,-0.00105,0.390,0.000,0.390,0.490,0.120,0.500
2,3,10811496,K00753.01,NaN,CANDIDATE,Done,2018-08-16,CANDIDATE,0.000,0,...,-0.021,-0.038,0.00070,0.00060,-0.025,-0.034,0.042,0.002,-0.027,0.027
3,4,10848459,K00754.01,NaN,FALSE POSITIVE,Done,2018-08-16,FALSE POSITIVE,0.000,0,...,-0.111,0.002,0.00302,-0.00142,-0.249,0.147,0.289,-0.257,0.099,0.276
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,Done,2018-08-16,CANDIDATE,1.000,0,...,-0.010,0.230,0.00008,-0.00007,0.030,-0.090,0.100,0.070,0.020,0.070


## 3. Inspección inicial del dataset original

En esta sección se realiza una primera exploración del dataset original:

- Se cargan todas las filas y columnas del archivo KOI.
- Se revisan las dimensiones del dataset (número de filas y columnas).
- Se muestra una vista previa (`head`) para entender la estructura de los datos.
- Se inspeccionan tipos de datos (`dtypes`) y algunas columnas clave como `koi_disposition`.

El objetivo es entender qué información tenemos antes de aplicar cualquier filtro o transformación.


In [ ]:
# Listamos las primeras columnas para tener una idea de la estructura
print("Primeras 30 columnas:")
print(df_raw.columns[:30].tolist())

print("\nNúmero total de columnas:", len(df_raw.columns))

# Revisamos los posibles valores de koi_disposition
print("\nValores únicos de koi_disposition:")
print(df_raw['koi_disposition'].value_counts())

Primeras 30 columnas:
['rowid', 'kepid', 'kepoi_name', 'kepler_name', 'koi_disposition', 'koi_vet_stat', 'koi_vet_date', 'koi_pdisposition', 'koi_score', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec', 'koi_disp_prov', 'koi_comment', 'koi_period', 'koi_time0bk', 'koi_time0', 'koi_eccen', 'koi_longp', 'koi_impact', 'koi_duration', 'koi_ingress', 'koi_depth', 'koi_ror', 'koi_srho', 'koi_fittype', 'koi_prad', 'koi_sma', 'koi_incl']

Número total de columnas: 83

Valores únicos de koi_disposition:
koi_disposition
FALSE POSITIVE    4839
CONFIRMED         2746
CANDIDATE         1979
Name: count, dtype: int64


## 4. Filtrar solo exoplanetas confirmados y falsos positivos

La columna `koi_disposition` indica el estado de cada candidato (`CONFIRMED`, `FALSE_POSITIVE`, `CANDIDATE`, etc.).

En esta fase se:

- Conservan solo las filas con `koi_disposition` igual a `CONFIRMED` o `FALSE_POSITIVE`.
- Se descartan los casos ambiguos o aún no confirmados.
- Se cuenta cuántos objetos hay en cada categoría para verificar el balance de clases.

Con esto definimos claramente el problema como una **clasificación binaria**.


In [ ]:
# Definimos las disposiciones que nos interesan
allowed_dispositions = ['CONFIRMED', 'FALSE POSITIVE']

# Filtramos el dataset
df = df_raw[df_raw['koi_disposition'].isin(allowed_dispositions)].copy()

print("Dimensiones del dataset FILTRADO (solo CONFIRMED/FALSE POSITIVE):", df.shape)
print("\nFrecuencia de cada disposición en el dataset filtrado:")
print(df['koi_disposition'].value_counts())
print("\nProporciones (para ver el desbalance):")
print(df['koi_disposition'].value_counts(normalize=True))

Dimensiones del dataset FILTRADO (solo CONFIRMED/FALSE POSITIVE): (7585, 83)

Frecuencia de cada disposición en el dataset filtrado:
koi_disposition
FALSE POSITIVE    4839
CONFIRMED         2746
Name: count, dtype: int64

Proporciones (para ver el desbalance):
koi_disposition
FALSE POSITIVE    0.63797
CONFIRMED         0.36203
Name: proportion, dtype: float64


## 5. Crear la variable objetivo binaria `is_confirmed`

Para facilitar el uso de modelos de clasificación, se crea una nueva variable objetivo:

- `is_confirmed = 1` si `koi_disposition == "CONFIRMED"`.
- `is_confirmed = 0` si `koi_disposition == "FALSE_POSITIVE"`.

Esta columna será la etiqueta que los modelos intentarán predecir en las fases de modelado supervisado.


In [ ]:
# y = 1 si koi_disposition == 'CONFIRMED'
# y = 0 si koi_disposition == 'FALSE POSITIVE'

df['is_confirmed'] = (df['koi_disposition'] == 'CONFIRMED').astype(int)

print("Ejemplo de la nueva variable objetivo:")
print(df[['koi_disposition', 'is_confirmed']].head())

print("\nConteo de la variable is_confirmed (y):")
print(df['is_confirmed'].value_counts())
print("\nProporciones de is_confirmed:")
print(df['is_confirmed'].value_counts(normalize=True))

Ejemplo de la nueva variable objetivo:
  koi_disposition  is_confirmed
0       CONFIRMED             1
1       CONFIRMED             1
3  FALSE POSITIVE             0
4       CONFIRMED             1
5       CONFIRMED             1

Conteo de la variable is_confirmed (y):
is_confirmed
0    4839
1    2746
Name: count, dtype: int64

Proporciones de is_confirmed:
is_confirmed
0    0.63797
1    0.36203
Name: proportion, dtype: float64


## 6. Seleccionar columnas útiles y eliminar columnas no informativas

El dataset original contiene:

- Identificadores internos.
- Columnas administrativas.
- Variables con muchos valores faltantes.
- Atributos que no aportan información física relevante para el modelo.

En este bloque se:

- Define un conjunto de columnas que **sí** son útiles (parámetros físicos del sistema estrella–planeta, banderas de falsos positivos, etc.).
- Se eliminan columnas redundantes, administrativas o con demasiados nulos.
- Se verifica cuántas columnas finales se mantienen para el análisis posterior.

El resultado es un subconjunto de variables más compacto y manejable.


In [ ]:
# Queremos quedarnos con variables numéricas/categóricas codificables,
# y eliminar:
# - IDs (rowid, kepid, kepoi_name, kepler_name)
# - la columna original koi_disposition (ya está codificada en is_confirmed)
# - campos de texto libre, comentarios, URLs, etc.

cols_to_drop = [
    'rowid',          # índice interno
    'kepid',          # identificador numérico de estrella
    'kepoi_name',     # nombre KOI (texto)
    'kepler_name',    # nombre Kepler (texto)
    'koi_disposition',# ya la mapeamos a is_confirmed
    'koi_vet_stat',   # estado de vetting (texto)
    'koi_vet_date',   # fecha de vetting (texto)
    'koi_pdisposition', # disposición previa (texto)
    'koi_disp_prov',  # proveedor de la disposición (texto)
    'koi_comment',    # comentarios (texto libre)
    'koi_fittype',    # tipo de ajuste (texto)
    'koi_limbdark_mod', # modelo limb darkening (texto)
    'koi_parm_prov',  # proveedor de parámetros (texto)
    'koi_tce_delivname', # nombre del delivery TCE (texto)
    'koi_quarters',   # descripción de quarters (texto/cadena compleja)
    'koi_trans_mod',  # modelo de tránsito (texto)
    'koi_datalink_dvr', # enlaces de data validation (URLs)
    'koi_datalink_dvs', # enlaces de data validation summary (URLs)
    'koi_sparprov'    # proveedor de parámetros estelares (texto)
]

# Nos aseguramos de que solo intentamos eliminar columnas que realmente existen
cols_to_drop = [c for c in cols_to_drop if c in df.columns]

print("Columnas a eliminar (no útiles o texto):")
print(cols_to_drop)

# Eliminamos estas columnas
df_clean = df.drop(columns=cols_to_drop).copy()

print("\nDimensiones después de eliminar columnas no útiles:", df_clean.shape)

# Comprobamos los tipos de datos resultantes
print("\nResumen de tipos de datos tras limpieza:")
print(df_clean.dtypes.value_counts())

Columnas a eliminar (no útiles o texto):
['rowid', 'kepid', 'kepoi_name', 'kepler_name', 'koi_disposition', 'koi_vet_stat', 'koi_vet_date', 'koi_pdisposition', 'koi_disp_prov', 'koi_comment', 'koi_fittype', 'koi_limbdark_mod', 'koi_parm_prov', 'koi_tce_delivname', 'koi_quarters', 'koi_trans_mod', 'koi_datalink_dvr', 'koi_datalink_dvs', 'koi_sparprov']

Dimensiones después de eliminar columnas no útiles: (7585, 65)

Resumen de tipos de datos tras limpieza:
float64    59
int64       6
Name: count, dtype: int64


## 7. Resumen de valores faltantes tras la limpieza básica

Después de seleccionar las columnas relevantes, es importante revisar:

- Qué proporción de valores faltantes queda en cada variable.
- Si alguna columna sigue teniendo demasiados nulos como para ser útil.
- Si es necesario planificar imputación o eliminación adicional en fases posteriores.

Este resumen sirve como puente hacia el EDA y el preprocesamiento más detallado de la Fase 2.


In [ ]:
# Esto nos sirve para el reporte y para planear la siguiente fase (imputación/limpieza adicional).

missing_count = df_clean.isnull().sum()
missing_pct = df_clean.isnull().mean() * 100  # porcentaje

missing_summary = (
    pd.DataFrame({
        'missing_count': missing_count,
        'missing_pct': missing_pct
    })
    .sort_values(by='missing_pct', ascending=False)
)

print("Resumen de valores faltantes (top 15 variables con más NaN):")
missing_summary.head(15)

Resumen de valores faltantes (top 15 variables con más NaN):


,missing_count,missing_pct
koi_ingress,7585,100.000000
koi_sage,7585,100.000000
koi_longp,7585,100.000000
koi_model_chisq,7585,100.000000
koi_model_dof,7585,100.000000
koi_fwm_stat_sig,962,12.682927
koi_score,910,11.997363
koi_bin_oedp_sig,910,11.997363
koi_fwm_prao,746,9.835201
koi_fwm_pdeco,718,9.466051


## 8. Resumen y conclusión de la Fase 1

En resumen, en esta fase hemos:

- Partimos del catálogo KOI original, con múltiples tipos de candidatos.
- Filtrado únicamente los objetos con disposición `CONFIRMED` y `FALSE_POSITIVE`, formulando un problema de clasificación binaria.
- Creado la variable objetivo `is_confirmed` (1 = exoplaneta confirmado, 0 = falso positivo).
- Seleccionado un subconjunto de columnas físicas y banderas relevantes, eliminando atributos administrativos o poco informativos.
- Revisado los valores faltantes para tener una idea clara de la calidad de los datos.

El resultado es un dataset más limpio y enfocado, que servirá como entrada directa para el análisis exploratorio y el preprocesamiento estadístico de la **Fase 2**.


In [ ]:
n_rows, n_cols = df_clean.shape
print(f"Filas (instancias): {n_rows}")
print(f"Columnas totales: {n_cols}")

# Asumiendo que 'is_confirmed' es nuestra y y el resto son X
n_features = n_cols - 1  # todas menos la variable objetivo
print(f"Características (X): {n_features}")
print("Variable objetivo (y): 'is_confirmed' (1 = CONFIRMED, 0 = FALSE POSITIVE)")

# Vemos las primeras filas para comprobar que todo está razonable
df_clean.head()

Filas (instancias): 7585
Columnas totales: 65
Características (X): 64
Variable objetivo (y): 'is_confirmed' (1 = CONFIRMED, 0 = FALSE POSITIVE)


,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_time0,koi_eccen,koi_longp,...,koi_fwm_sdeco,koi_fwm_prao,koi_fwm_pdeco,koi_dicco_mra,koi_dicco_mdec,koi_dicco_msky,koi_dikco_mra,koi_dikco_mdec,koi_dikco_msky,is_confirmed
0,1.000,0,0,0,0,9.488036,170.538750,2455003.539,0.0,NaN,...,0.940,-0.00020,-0.00055,-0.010,0.200,0.200,0.080,0.310,0.320,1
1,0.969,0,0,0,0,54.418383,162.513840,2454995.514,0.0,NaN,...,1.230,0.00066,-0.00105,0.390,0.000,0.390,0.490,0.120,0.500,1
3,0.000,0,1,0,0,1.736952,170.307565,2455003.308,0.0,NaN,...,0.002,0.00302,-0.00142,-0.249,0.147,0.289,-0.257,0.099,0.276,0
4,1.000,0,0,0,0,2.525592,171.595550,2455004.596,0.0,NaN,...,0.230,0.00008,-0.00007,0.030,-0.090,0.100,0.070,0.020,0.070,1
5,1.000,0,0,0,0,11.094321,171.201160,2455004.201,0.0,NaN,...,0.140,0.00004,NaN,0.040,-0.070,0.080,-0.020,-0.080,0.080,1


## 9. Guardar el dataset limpio para las siguientes fases

Finalmente, se guarda el dataset resultante en un archivo CSV:

- Nombre del archivo: `exoplanet_koi_binary_clean.csv`.
- Este archivo será cargado en la Fase 2 para realizar el EDA (análisis exploratorio).
- También será la base para la ingeniería de características y el modelado supervisado en las fases posteriores.

De esta forma aseguramos que todas las fases usan exactamente la misma versión “congelada” del dataset limpio.


In [ ]:
output_path = "/content/exoplanet_koi_binary_clean.csv"

# Guardamos sin el índice
df_clean.to_csv(output_path, index=False)

print("Dataset limpio guardado en:", output_path)
print("Dimensiones finales:", df_clean.shape)

Dataset limpio guardado en: /content/exoplanet_koi_binary_clean.csv
Dimensiones finales: (7585, 65)
